In [5]:
import os
import pickle
os.chdir('../')
from mtDNAsim import *

/home/wangkun/mtDNA_simulation/mtDNAsim/sim_utils.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [91]:
path = '/data3/wangkun/mtsim_res/240705/linear_const'
simid = '5561'

In [101]:
gen, p = 100, 0.1
gt_tree = loadtree(f'{path}/{simid}/clonal_expansion_tree_{p}_{gen}.nwk')[0]
for i in gt_tree.get_terminals():
    if i.name.count('_')==1:
        i.name = f'{i.name}_0'

In [94]:
mts = pickle.load(open(f'{path}/{simid}/mt_allmuts_const_0.1_{simid}_{gen}_0.4_{p}_rs.pkl', 'rb'))
mts = rs_cvt(mts)
mt_freq = sparse_freq(mts, df=True)

  0%|          | 0/5542 [00:00<?, ?it/s]

In [ ]:
for gen in [100, 400]:
    for p in [0.1, 0.9]:
        mts = pickle.load(open(f'{path}/{simid}/mt_allmuts_const_0.1_{simid}_{gen}_0.4_{p}_rs.pkl', 'rb'))
        mts = rs_cvt(mts)
        mt_freq = sparse_freq(mts, df=True)
        gt_tree = loadtree(f'{path}/{simid}/clonal_expansion_tree_{p}_{gen}.nwk')[0]
        for i in gt_tree.get_terminals():
            if i.name.count('_')==1:
                i.name = f'{i.name}_0'
        mt_freq = mt_freq.loc[[i.name for i in gt_tree.get_terminals()]]
        for cutoff in [0, 0.01]:
            muts = mt_freq>cutoff
            muts = muts.astype(int).astype(str)
            translation_table = str.maketrans({'1': 'A', '0': 'G'})
            seqs = f'{mt_freq.shape[0]} {mt_freq.shape[1]}\n'
            for i in range(mt_freq.shape[0]):
                seqs += f'{mt_freq.index[i]} '
                seqs += ''.join(muts.iloc[i].to_numpy()).translate(translation_table)
                seqs += '\n'
            with open(f'{path}/{simid}/mt_allmuts_const_0.1_{simid}_{gen}_0.4_{p}_rs_{cutoff}.phy', 'w') as f:
                f.write(seqs)

In [ ]:
sl_script = [
'#!/bin/bash',
f'#SBATCH -J build_tree',
'#SBATCH -p all',
'#SBATCH -N 1',
'#SBATCH -n 1',
'#SBATCH --mem=10G',
'#SBATCH -t 0',
'#SBATCH -o oe/%x-%j.log ',
'#SBATCH -e oe/%x-%j.err' ]

In [63]:
mts = pickle.load(open(f'{path}/{simid}/mt_allmuts_const_0.1_{simid}_100_0.4_0.1_rs.pkl', 'rb'))
mts = rs_cvt(mts)
mt_freq = sparse_freq(mts, df=True)
muts = mt_freq>0
muts = muts.astype(int).astype(str)
translation_table = str.maketrans({'1': 'A', '0': 'G'})
seqs = f'{mt_freq.shape[0]} {mt_freq.shape[1]}\n'
for i in range(mt_freq.shape[0]):
    seqs += f'{mt_freq.index[i]} '
    seqs += ''.join(muts.iloc[i].to_numpy()).translate(translation_table)
    seqs += '\n'
with open(f'./scripts/mt_allmuts_const_0.1_{simid}_100_0.4_0.1_rs_0.phy', 'w') as f:
    f.write(seqs)

In [75]:
path = '/data3/wangkun/mtsim_res/240705/linear_const'
for i in os.listdir(path):
    for gen in [100,400]:
        for s in [0.1, 0.9]:
            tree = loadtree(f'{path}/{i}/clonal_expansion_tree_{s}_{gen}.nwk')[0]
            seqs = DNAmutation(tree, mut_rate=3.2)
            seqs = seqs.astype(int)
            with open(f'{path}/{i}/dna_mut3.2_{s}_{gen}.phy', 'w') as f:
                f.write('{} {}\n'.format(*seqs.shape))
                for cell in seqs.index:
                    f.write('{} {}\n'.format(cell[1:-1], ''.join(seqs.loc[cell].astype(str)).replace('0', 'A').replace('1', 'G')))

In [111]:
path = '/data3/wangkun/mtsim_res/240705/linear_'
for i in os.listdir(path):
    for gen in [100,400]:
        for s in [0.1, 0.9]:
            tree = loadtree(f'{path}/{i}/clonal_expansion_tree_{s}_{gen}.nwk')[0]
            seqs = DNAmutation(tree, mut_rate=0.8)
            seqs = seqs.astype(int)
            with open(f'{path}/{i}/dna_mut0.8_{s}_{gen}.phy', 'w') as f:
                f.write('{} {}\n'.format(*seqs.shape))
                for cell in seqs.index:
                    f.write('{} {}\n'.format(cell, ''.join(seqs.loc[cell].astype(str)).replace('0', 'A').replace('1', 'G')))

In [112]:
path = '/data3/wangkun/mtsim_res/240705/linear_const'
for i in os.listdir(path):
    for gen in [100,400]:
        for s in [0.1, 0.9]:
            tree = loadtree(f'{path}/{i}/clonal_expansion_tree_{s}_{gen}.nwk')[0]
            seqs = DNAmutation(tree, mut_rate=0.8)
            seqs = seqs.astype(int)
            with open(f'{path}/{i}/dna_mut0.8_{s}_{gen}.phy', 'w') as f:
                f.write('{} {}\n'.format(*seqs.shape))
                for cell in seqs.index:
                    f.write('{} {}\n'.format(cell, ''.join(seqs.loc[cell].astype(str)).replace('0', 'A').replace('1', 'G')))

In [88]:
def DNAmutation(tree, mut_rate=0.1):
    mutations = dict()
    global_mutid = -1
    for i in tree.get_terminals():
        mut = []
        for j in tree.get_path(i):
            if j in mutations:
                mut = deepcopy(mutations[j])
            else:
                for _ in range(np.random.poisson(mut_rate)):
                    mut.append(global_mutid+1)
                    global_mutid += 1
                mutations[j] = deepcopy(mut)

    mut_table = []
    cell_names = []
    for i in tree.get_terminals():
        seq = np.zeros(global_mutid+1)
        seq[mutations[i]]=1
        mut_table.append(seq)
        cell_names.append(i.name)
    return pd.DataFrame(np.array(mut_table), index=cell_names)

In [87]:
tree.get_path(tree.get_terminals()[0])

[Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, confidence=1.0),
 Clade(branch_length=1.0, name='<18_3508>_24')]

In [67]:
tree = loadtree('/data3/wangkun/mtsim_res/240705/linear_/841652/clonal_expansion_tree_0.9_400.nwk')[0]

In [72]:
seqs = DNAmutation(tree, mut_rate=3.2)

In [68]:
seqs = DNAmutation(tree, mut_rate=3.2)
seqs = seqs.astype(int)

with open(f'../results/dna_mut0.5_{filename}.phy', 'w') as f:
    f.write('{} {}\n'.format(*seqs.shape))
    for cell in seqs.index:
        f.write('{} {}\n'.format(cell, ''.join(seqs.loc[cell].astype(str)).replace('0', 'A').replace('1', 'G')))
        

Tree(rooted=False, weight=1.0)

In [78]:
cell

'<8_194>_5'

In [ ]:
os.system(f'Rscript tree_reconstruct.r -f {fn} -p {path}')